In [29]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [30]:
#bring in the data file 
data = Path('train.csv')
titanic_df = pd.read_csv(data, index_col=0)
titanic_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [31]:
#drop NaaNs
titanic_df = titanic_df.dropna()
titanic_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [32]:
#drop unnecessary data fields
titanic_df = titanic_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
2,1,1,female,38.0,1,0,71.2833,C
4,1,1,female,35.0,1,0,53.1000,S
7,0,1,male,54.0,0,0,51.8625,S
11,1,3,female,4.0,1,1,16.7000,S
12,1,1,female,58.0,0,0,26.5500,S


In [33]:
#encode non-numerical values 
titanic_encoded = pd.get_dummies(titanic_df, columns=
    ["Sex", 
     "Embarked" 
    ])
titanic_encoded.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,
2,1,1,38.0,1,0,71.2833,1,0,1,0,0
4,1,1,35.0,1,0,53.1000,1,0,0,0,1
7,0,1,54.0,0,0,51.8625,0,1,0,0,1
11,1,3,4.0,1,1,16.7000,1,0,0,0,1
12,1,1,58.0,0,0,26.5500,1,0,0,0,1


In [34]:
# Segment the features from the target
X = titanic_encoded.drop("Survived", axis=1)
y = titanic_encoded['Survived']

In [35]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [36]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [37]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [38]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [39]:
# Create the random forest classifier instance
titanic_rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [40]:
# Fit the model
titanic_rf_model = titanic_rf_model.fit(X_train_scaled, y_train)

In [41]:
# Making predictions using the testing data
rf_predictions = titanic_rf_model.predict(X_test_scaled)

In [42]:
# Calculate the confusion matrix
rf_cm = confusion_matrix(y_test, rf_predictions)
rf_cm_df = pd.DataFrame(
    rf_cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)


In [43]:
# Calculating the accuracy score

acc_score = accuracy_score(y_test, rf_predictions)

In [45]:
# Displaying results
print("Confusion Matrix")
display(rf_cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, rf_predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,7,5
Actual 1,8,26


Accuracy Score : 0.717391304347826
Classification Report
              precision    recall  f1-score   support

           0       0.47      0.58      0.52        12
           1       0.84      0.76      0.80        34

    accuracy                           0.72        46
   macro avg       0.65      0.67      0.66        46
weighted avg       0.74      0.72      0.73        46



## Feature Importance 

In [46]:
# Get the feature importance array
importances = titanic_rf_model.feature_importances_

In [48]:
# List the features from most to least important
importances_sorted = sorted(zip(titanic_rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.3342002174297532, 'Age'),
 (0.260381279721028, 'Fare'),
 (0.14300739607640814, 'Sex_male'),
 (0.1340980201844561, 'Sex_female'),
 (0.04293204114038206, 'Parch'),
 (0.027599420380507382, 'SibSp'),
 (0.0248884909168144, 'Pclass'),
 (0.015713145839721297, 'Embarked_C'),
 (0.015080922147341547, 'Embarked_S'),
 (0.002099066163587874, 'Embarked_Q')]